In [8]:
import json
from make_taxonomy_rdrs import *
from get_ner_from_query2 import *
from datasets.process_rdrs import *

In [9]:
# distinc_entities_dict = get_distinc_entity_dict("datasets/rdrs/rdrs")
# tag_topic_freq = {"Medication":{}, "Disease":{}, "ADR":{}}

# with open("retrived_topics/rdrs/rdrs_entity_topics_dict.json", "r", encoding="utf-8") as reader:
#     data = json.load(reader)
#     for entity, topics in data.items():
#         for topic, id_topic in topics.items():
#             ent_tag_freq_dict = distinc_entities_dict[entity] # distinc_entities_dict == {entity : {tag1:freq, tag2:freq, ...}}
#             topic = f"{topic} | {id_topic}"
#             for tag, freq in ent_tag_freq_dict.items():
#                 if topic not in tag_topic_freq[tag]:
#                     tag_topic_freq[tag][topic] = freq
#                 else:
#                     tag_topic_freq[tag][topic] += freq
#         #print(tag_topic_freq)

# for tag, topics in tag_topic_freq.items():
#     topics = dict(sorted(topics.items(), key=lambda x: x[1], reverse=True))
#     write_to_file(f"retrived_topics/rdrs/{tag}_topic_freq.txt", topics)

In [10]:
def add_to_tag_topic_entity_dict(taxonomy_file, LIMIT, LANG, rdrs_topic_entity_dict):
    filename_with_extension = os.path.basename(taxonomy_file)
    tag_name = os.path.splitext(filename_with_extension)[0]
    itemLabel_set = {}
    taxonomy_ids = []
    with open(taxonomy_file, "r", encoding="utf-8") as reader:
        lines = reader.readlines()
        for line in lines:
            tax_id = line.split()[-1]
            taxonomy_ids.append(tax_id)


    for tax_id in tqdm(taxonomy_ids, desc=f"Creating gzt for {tag_name}"):
        current_limit = LIMIT
        results = {}
        while current_limit > 0:
            try:
                results = get_results(endpoint_url, current_limit, tax_id, LANG)
                break  # If successful, break out of the while loop
            except Exception as e:
                print(f"Error fetching results with limit {current_limit}: {e}")
                if current_limit <= 500000:
                    current_limit -= 10000  # Decrease limit by 10,000 and retry
                else:
                    current_limit /= 2
                if current_limit <= 0:
                    print(f"Failed to fetch results for tax_id {tax_id} after several attempts.")
                    continue  # Skip to the next tax_id if limit becomes zero or negative
        
        if len(results) > 0:
            for result in results["results"]["bindings"]:
                itemLabel = result['itemLabel']['value'].lower()
                if tax_id not in rdrs_topic_entity_dict:
                    rdrs_topic_entity_dict[tax_id] = [itemLabel]
                else:
                    rdrs_topic_entity_dict[tax_id].append(itemLabel)

In [11]:
# taxonomy_directory = 'label_taxonomy/rdrs/'  # Specify the directory path
# LIMIT = 1000000
# LANG = "ru"

# for filename in os.listdir(taxonomy_directory):
#     if filename.endswith(".txt"):
#         tag_name = os.path.splitext(filename)[0]
#         tag_topic_entity_dict = {}
#         filepath = os.path.join(taxonomy_directory, filename)
#         print(f"Processing {filepath}")
#         add_to_tag_topic_entity_dict(filepath, LIMIT, LANG, tag_topic_entity_dict)
#         write_to_file(f"rdrs_{tag_name}_entity_dict.json", tag_topic_entity_dict)

In [12]:
# def read_from_file(file_path):
#     with open(file_path, 'r', encoding='utf-8') as f:
#         data = json.load(f)
#     return data

# def write_list_of_words_to_file_txt(filepath, mylist):
#     with open(filepath, "w", encoding="utf-8") as writer:
#         for word in mylist:
#             writer.write(f"{word}\n")

# fold = "rdrs5"
# train_file = f"datasets/rdrs/{fold}"
# retrived_topic_dict = read_from_file("retrived_topics/rdrs/rdrs_entity_topics_dict.json")
# ADR_topic_dict = read_from_file("label_taxonomy/rdrs/rdrs_ADR_entity_dict.json")
# Medication_topic_dict = read_from_file("label_taxonomy/rdrs/rdrs_Medication_entity_dict.json")
# Disease_topic_dict = read_from_file("label_taxonomy/rdrs/rdrs_Disease_entity_dict.json")

# ADR_entity_list = set()
# Medication_entity_list = set()
# Disease_entity_list = set()

# distinc_entity_dict = get_distinc_entity_dict(train_file)
# topic_dict_train_file = set()
# for entity in distinc_entity_dict:
#     topic_dict = retrived_topic_dict[entity]
#     for topic, id in topic_dict.items():
#         topic_dict_train_file.add(id)
# for id in topic_dict_train_file:
#     if id in ADR_topic_dict:
#         for entity in ADR_topic_dict[id]:
#             ADR_entity_list.add(entity)
#     if id in Medication_topic_dict:
#         for entity in Medication_topic_dict[id]:
#             Medication_entity_list.add(entity)
#     if id in Disease_topic_dict:
#         for entity in Disease_topic_dict[id]:
#             Disease_entity_list.add(entity)

# os.makedirs(f"gazetteers/gzt_{fold}/", exist_ok=True)
# write_list_of_words_to_file_txt(f"gazetteers/gzt_{fold}/ADR.txt", ADR_entity_list)
# write_list_of_words_to_file_txt(f"gazetteers/gzt_{fold}/Medication.txt", Medication_entity_list)
# write_list_of_words_to_file_txt(f"gazetteers/gzt_{fold}/Disease.txt", Disease_entity_list)

In [19]:
fold = 5
freq_labels = {}

process_data(f"/home/andrew6072/Downloads/dataset/RDRS-main/data/interim/{fold}/valid.json", f"rdrs{fold}_valid.txt", freq_labels)
with open(f"datasets/rdrs/rdrs{fold}_valid_freq.txt", 'w', encoding='utf-8') as writer:
    for label, freq in freq_labels.items():
        writer.write(f"{label} {freq}\n")

Processing rdrs5_valid.txt: 100%|██████████| 327/327 [00:05<00:00, 55.83it/s]
